In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# these three imports are aliased as np, pd, and plt by convention
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# other imports can go here
from datetime import datetime

In [3]:
df = pd.read_csv('revs_scored_jan.tsv', sep='\t', header=0)

In [4]:
print(df)

         rev_timestamp   page_id     rev_id prev_rev_id  is_minor  \
0           1546300800  47649229  876227289   869530001      True   
1           1546300801  47642850  876227292   869530006      True   
2           1546300801  59537875  876227290        None     False   
3           1546300801  60902958  876227291   869530004      True   
4           1546300802  31392285  876227293   876069768      True   
...                ...       ...        ...         ...       ...   
4766347     1548979197   1904845  881187547   868089901      True   
4766348     1548979198  24973891  881187550   881177866     False   
4766349     1548979198  47592150  881187546   881187304     False   
4766350     1548979198  57420342  881187549   881187524      True   
4766351     1548979199  59823847  881187551   881187083     False   

              user_text   user_id seconds_to_prev  curr_bytes delta_bytes  \
0                BD2412    196446         3696519        1195          -9   
1                



Bot revisions are those where `user_is_bot` is true. Question: How many bots made edits in January 2019? (Hint: The name of 
the user who made the revision is given in the user_text column. I am asking: how many unique 'user_text' values are there 
for rows where user_is_bot == True? There is a Pandas function that does this, but there are several reasonable approaches.)



In [7]:
df.user_is_bot.value_counts()

False    4241207
True      525145
Name: user_is_bot, dtype: int64

In [14]:
# number of unique bot ids

dfnew = df[df.user_is_bot]
len(dfnew.user_text.value_counts())

105

What % of bots edits are reverted and what % of human edits are reverted?  (Hint: This is already in the notebook.  Read the documentation for the crosstab command.  The column "is_reverted" tells you whether a revisions was reverted or not (according to a very particular discussion of "reverted", which we will discuss later).)


In [16]:
# percentage of bot edits reverted: 5%
# percentage of human edits reverted: 10%

pd.crosstab(df.user_is_bot, df.is_reverted, margins=True, normalize='index').round(decimals=3)

is_reverted,False,True
user_is_bot,,
False,0.898,0.102
True,0.950,0.050
All,0.903,0.097


What bot has the most revisions in the dataset? (Hint: already in the notebook. See: pd.DataFrame.value_counts.) What does Cydebot do?  Try to find the original bot approval discussion for Cydebot. (Hint: read about the bot approvals process, find the archived bot approvals list, look for Cydebot. Extra credit: Is Cydebot still running as of June 2021?  If not, what happened to it and why?)

In [17]:
dfnew.user_text.value_counts().head(1)

Cydebot    42742
Name: user_text, dtype: int64

Cydebot looks at discussions about category deletion, then automatically goes through and deletes mentions of those pages after discussions are concluded.

https://en.wikipedia.org/wiki/Special:Contributions/Cydebot
https://en.wikipedia.org/wiki/Wikipedia:Categories_for_discussion/Log/2020_January_2#Bunch_of_journal/magazine_people_categories

Cydebot was banned because it was deleting things even though no consensus was reached in the discussion (linked).

https://en.wikipedia.org/wiki/Wikipedia:Bots/Requests_for_approval/Cydebot_4
^^ approval discussion